In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from IPython.display import Image
from qutip import *
from math import sqrt


from qutip.qip.device import Processor
from qutip_qip.device import CavityQEDModel
from qutip.qip.compiler import CavityQEDCompiler
from qutip.qip.pulse import Pulse
from qutip.operators import sigmaz, sigmay, sigmax, destroy
from qutip.states import basis
from qutip.metrics import fidelity
from qutip.qip.operations import rx, ry, rz, hadamard_transform

In [5]:
GHz = 10**9 #Hz
wq1 = 3.04 * 2* np.pi#* GHz # 6.617 * GHz * 2* np.pi # wq = sqrt(8EjEc)
wq2 = 3.00 * 2* np.pi#* GHz # 2.88 * GHz * 2* np.pi # 6.529 * GHz * 2* np.pi
wc = 3 * 2 * np.pi #* GHz # 5.19 * GHz * 2 * np.pi
g1 = 0.04 * 2 * np.pi #* GHz  # 0.105 * GHz * 2 * np.pi
g2 = 0.04 * 2 * np.pi #* GHz # g1
# cavity dissapation rate
kappa = 0#0.033 * 2 * np.pi * GHz
# qubit dissapation rate
gamma1 = 0.0#0.00261 * GHz * 2* np.pi
gamma2 = gamma1

In [7]:
# https://notebook.community/qutip/qutip-notebooks/examples/qip-noisy-device-simulator
# still don't know how to add detuning of qubits
processor = CavityQEDModel(num_qubits=2, num_levels=10, w0=wc, g=[g1,g2])

In [9]:
psi0 = tensor(basis(10,1), basis(2,0), basis(2,0))
result = processor.run(init_state=psi0)

AttributeError: 'CavityQEDModel' object has no attribute 'run'